# Code below import the data from local disk

In [170]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

from functools import reduce

In [87]:
# load the train and test data
trainInput = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/train_in.csv')
trainOutput = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/train_out.csv')
testInput = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/test_in.csv')
testOutput = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/test_out.csv')

In [88]:
# Check the dimension
print("Dimension of train data input: {}".format(trainInput.shape))
print("Dimension of train data output: {}".format(trainOutput.shape))
print("Dimension of test data input: {}".format(testInput.shape))
print("Dimension of test data output: {}".format(testOutput.shape))

Dimension of train data input: (304661, 1001)
Dimension of train data output: (304661, 12)
Dimension of test data input: (1200, 1001)
Dimension of test data output: (1200, 12)


In [89]:
# extract the modification class
modification_class = trainOutput.columns.tolist()
print(modification_class)

# check the proportion of dataset for each class from trainOutput
total_modified_data = 0
for cls in modification_class:
    modified_data_count = trainOutput[cls].sum()
    total_modified_data += modified_data_count
    print("class %s : %.4f" %(cls, modified_data_count / trainOutput.shape[0] * 100) + "%")
print("total modified data: %.4f" %(total_modified_data / trainOutput.shape[0] * 100) + "%")

['hAm', 'hCm', 'hGm', 'hTm', 'hm1A', 'hm5C', 'hm5U', 'hm6A', 'hm6Am', 'hm7G', 'hPsi', 'Atol']
class hAm : 0.4566%
class hCm : 0.5508%
class hGm : 0.4172%
class hTm : 0.6739%
class hm1A : 5.2997%
class hm5C : 0.9870%
class hm5U : 1.1475%
class hm6A : 21.3280%
class hm6Am : 0.7375%
class hm7G : 0.2744%
class hPsi : 0.9640%
class Atol : 17.2054%
total modified data: 50.0418%


In [92]:
# join the input and output data
trainFull = trainInput.join(trainOutput)
testFull = testInput.join(testOutput)
print("full train dataset: \n" + str(trainFull.head()))

full train dataset: 
  V1 V2 V3 V4 V5 V6 V7 V8 V9 V10  ...  hGm  hTm hm1A hm5C hm5U hm6A hm6Am  \
0  C  C  C  C  A  T  A  C  C   C  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
1  T  C  T  C  C  T  G  C  C   T  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
2  C  A  G  A  T  A  G  T  A   A  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
3  T  G  T  C  T  T  T  T  A   C  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
4  A  T  T  A  C  T  T  A  A   T  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0   

  hm7G hPsi Atol  
0  0.0  0.0  0.0  
1  0.0  0.0  0.0  
2  0.0  0.0  0.0  
3  0.0  0.0  0.0  
4  0.0  0.0  0.0  

[5 rows x 1013 columns]


In [117]:
# add a target column for easy identification of binary class of the modification class (yes or no)
trainFull['target'] = trainFull[modification_class].sum(axis=1)
testFull['target'] = testFull[modification_class].sum(axis=1)

# Code below make use of modified train dataset to split it into 12 subsets

In [118]:
# define a function to split the modified data to 12 sub classes
def split_into_12ModifiedClass(data, sub_classes_list):
    outputDic = {}
    for cls in modification_class:
        drop_irrelevant = [c for c in modification_class if c!= cls]
        outputDic[cls] = data[data[cls] == 1].drop(columns=drop_irrelevant).rename(columns={cls:"target"}) 
    return outputDic

In [119]:
# split the datasets into 12 subsets of modified classes
trainFull_modifiedDic = split_into_12ModifiedClass(trainFull, modification_class)

In [120]:
# check the size of the subsets
for cls in modification_class:
    print("Data size for class %s: %d" %(cls, trainFull_dic[cls].shape[0]))

Data size for class hAm: 1391
Data size for class hCm: 1678
Data size for class hGm: 1271
Data size for class hTm: 2053
Data size for class hm1A: 16146
Data size for class hm5C: 3007
Data size for class hm5U: 3496
Data size for class hm6A: 64978
Data size for class hm6Am: 2247
Data size for class hm7G: 836
Data size for class hPsi: 2937
Data size for class Atol: 52418


In [121]:
# display a sample data
df = trainFull_modifiedDic[modification_class[1]]
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V994,V995,V996,V997,V998,V999,V1000,V1001,target,target
2782,T,T,T,C,T,T,C,A,T,G,...,C,T,G,C,C,C,C,C,1.0,1.0
2783,C,T,T,T,T,T,T,T,T,T,...,A,T,G,G,A,G,T,C,1.0,1.0
2784,A,G,T,A,G,G,C,C,C,A,...,C,T,T,T,C,A,T,C,1.0,1.0
2785,G,G,G,A,G,G,T,T,T,G,...,T,G,T,A,T,T,A,A,1.0,1.0
2786,G,G,T,C,A,T,G,C,C,T,...,T,A,A,C,A,A,A,C,1.0,1.0


In [128]:
# retrieve the index of the modified-groups based on the 12 subsets formed
# this will help to split the entire train dataset into 2 major groups of modified and non-modified
index_modifiedData = [v.index.tolist() for k,v in trainFull_modifiedDic.items()]
flatten_index_modifiedData = [idx for index_set in index_modifiedData for idx in index_set]
len(flatten_index_modifiedData)

# split into 2 major groups of data
# drop the individual modification class column to reduce the dataset size
trainFull_MD = trainFull[trainFull.index.isin(flatten_index_modifiedData)].drop(columns=modification_class).reset_index(drop=True)
trainFull_NMD = trainFull[trainFull.index.isin(list(set(trainFull.index)-set(flatten_index_modifiedData)))].drop(columns=modification_class).reset_index(drop=True)

In [129]:
# check the dimension of data
print("modified dataset size: " + str(trainFull_MD.shape))
print("non-modified dataset size: " + str(trainFull_NMD.shape))
print("total dataset size: " + str(trainFull_MD.shape[0]+ trainFull_NMD.shape[0]))

modified dataset size: (152453, 1002)
non-modified dataset size: (152208, 1002)
total dataset size: 304661


# Code below run test by randomly picking an input from the test dataset
# Then compute #hits based on position-to-position match

In [239]:
# define a function to visualize statistical data for different sample poolsize (e.g. 100, 1000, 2000, 5000, 10000)
def compute_statistical_data(sample_size):
    
    # randomly pick a test sample for computation purpose
    sample_input = testFull.sample(n=1).drop(columns=modification_class)
    sample_input_type = "is modified" if (sample_input['target'].get('target') == 1) else "is non-modified"
    print("selected test sample " + sample_input_type)
    print("-"*50)
    
    # define a dictionary each to hold the hits count against each train sample
    hitMD = {}
    hitNMD = {}
    
    sample_MD = []
    sample_NMD = []
    # check hits count through position-position check
    for size in sample_size:
        
        # randomly create a sample pool for each group
        modified_data = trainFull_MD.sample(n=size, replace=False)
        non_modified_data = trainFull_NMD.sample(n=size, replace=False)
        
        # append the data to the sample list
        sample_MD.append(modified_data)
        sample_NMD.append(non_modified_data)
        
    # compute the hits statistics against modified pool
    print("against modified pool:\n")    
    for data in sample_MD:
        size = data.shape[0]
        for idx, row in data.iterrows():
            hit = 0
            for col, data in row.iteritems():
                if (col != 'target') & (demo_input[col].values[0] == data):
                    hit += 1
            hitMD[idx] = hit

        top10_hitMD = sorted(hitMD.items(), key=lambda kv : kv[1], reverse=True)[:10]
        #print("top10 hits:\n" + str(top10_hitMD) + "\n")
        hitMD_list = [kv[1] for kv in hitMD.items()]
        avg_hits = reduce(lambda a,b : a+b, hitMD_list) / len(hitMD_list)
        std_hits = np.std(hitMD_list, ddof=1)
        print("n=%d, " %size + "avg hits:%.2f, " %avg_hits + "std hits:%.2f " %std_hits)
     
    print("-"*50)
    
    print("against non-modified pool:\n")
    for data in sample_NMD:
        size = data.shape[0]
        for idx, row in data.iterrows():
            hit = 0
            for col, data in row.iteritems():
                if (col != 'target') & (demo_input[col].values[0] == data):
                    hit += 1
            hitNMD[idx] = hit

        top10_hitNMD = sorted(hitNMD.items(), key=lambda kv : kv[1], reverse=True)[:10]
        #print("top10 hits:\n" + str(top10_hitNMD) + "\n")
        hitNMD_list = [kv[1] for kv in hitNMD.items()]
        avg_hits = reduce(lambda a,b : a+b, hitNMD_list) / len(hitNMD_list)
        std_hits = np.std(hitNMD_list, ddof=1)
        print("n=%d, " %size + "avg hits:%.2f, " %avg_hits + "std hits:%.2f" %std_hits)
        

In [241]:
# show statistical analysis
sample_size = [10, 100, 1000, 2000]
compute_statistical_data(sample_size)

selected test sample is non-modified
--------------------------------------------------
against modified pool:

n=10, avg hits:247.50, std hits:63.84 
n=100, avg hits:236.00, std hits:55.43 
n=1000, avg hits:232.03, std hits:55.05 
n=2000, avg hits:232.99, std hits:54.79 
--------------------------------------------------
against non-modified pool:

n=10, avg hits:194.60, std hits:50.47
n=100, avg hits:220.96, std hits:53.18
n=1000, avg hits:219.09, std hits:50.68
n=2000, avg hits:219.17, std hits:50.81
